In [9]:
#libraries
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [17]:
#get data from yahoo of any company
data_frame=web.DataReader('TSLA',data_source='yahoo',start='2010-01-01',end='2022-01-01')

In [20]:
#see the data
data_frame

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-06-29,5.000000,3.508000,3.800000,4.778000,93831500.0,4.778000
2010-06-30,6.084000,4.660000,5.158000,4.766000,85935500.0,4.766000
2010-07-01,5.184000,4.054000,5.000000,4.392000,41094000.0,4.392000
2010-07-02,4.620000,3.742000,4.600000,3.840000,25699000.0,3.840000
2010-07-06,4.000000,3.166000,4.000000,3.222000,34334500.0,3.222000
...,...,...,...,...,...,...
2021-12-27,1117.000000,1070.719971,1073.670044,1093.939941,23715300.0,1093.939941
2021-12-28,1119.000000,1078.420044,1109.489990,1088.469971,20108000.0,1088.469971
2021-12-29,1104.000000,1064.140015,1098.640015,1086.189941,18718000.0,1086.189941


In [22]:
#rows and columns
data_frame.shape

(2899, 6)